In [131]:
import time
import pandas as pd
import numpy as np

1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla
más fuerte que las demás.
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla
más fuerte que las demás.
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECO”
siendo esta regla más fuerte que la 1 y 2.
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de
mayor score a menor

In [132]:
df_pd = pd.read_csv('../data/Desafio Ds 2025 - Gen AI/ordenamiento.csv')

In [133]:
df_pd.head()

,item_id,vertical,category,domain,score
0,512208310,CPG,PETS FOOD,MLC-CATS_AND_DOGS_FOODS,0.0272
1,468513076,CE,ELECTRONICS,MLC-RANGES,0.9256
2,614337410,CE,ELECTRONICS,MLC-SMART_SPEAKERS,0.8304
3,634351318,APP & SPORTS,APPAREL,MLC-PANTS,0.0560
4,528383704,ACC,VEHICULAR MULTIMEDIA,MLC-GPS,0.2334


In [134]:
def algo_pd(df):
    start_time = time.time()

    df = df.sort_values(by='score', ascending=False).reset_index(drop=True)
    id3 = df[df['item_id'] == 641416750]
    id6 = df[df['item_id'] == 22351223]
    hd = df[(df['category'] == 'HOME&DECOR') & (df['score'] < df.iloc[:7]['score'].max())][:3]

    df = df[~df['item_id'].isin([641416750, 22351223] + list(hd['item_id']))]

    blacklist = pd.concat([hd, id3, id6])

    df = pd.concat([df.iloc[:2], id3, df.iloc[2:5], id6, df.iloc[5:7], hd, df[7:]]).reset_index(drop=True)

    for i in range(len(df) - 1):
        if df.iloc[i]['vertical'] == df.iloc[i+1]['vertical'] and blacklist[blacklist['item_id'] == df.iloc[i+1]['item_id']].empty:
            vert = df.iloc[i]['vertical']
            df_ok = df.iloc[i:][(df.iloc[i:]['vertical'] != vert) & (~df.iloc[i:]['item_id'].isin(hd['item_id']))].iloc[:2] if i != 6 \
                else df.iloc[i:][(df.iloc[i:]['vertical'] != vert) & (df.iloc[i:]['vertical'] != 'HOME & INDUSTRY') & (~df.iloc[i:]['item_id'].isin(hd['item_id']))].iloc[:2]

            df.iloc[i+1], df.iloc[df_ok.index[0]] = df.iloc[df_ok.index[0]], df.iloc[i+1]

        if (df.iloc[i]['domain'] == df.iloc[i:i+4]['domain'].unique()).all() and blacklist[blacklist['item_id'] == df.iloc[i+1]['item_id']].empty:  # Dada la regla 2, nunca deberia suceder la regla 1 salvo que se compartan domain entre verticales
            dom = df.iloc[i]['domain']
            df_ok = df.iloc[i:][(df.iloc[i:]['domain'] != dom) & (~df.iloc[i:]['item_id'].isin(hd['item_id']))].iloc[:2]
            df.iloc[i+4], df.iloc[df_ok.index[0]] = df.iloc[df_ok.index[0]], df.iloc[i+4]

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(elapsed_time)
    return df

In [135]:
df = algo_pd(df_pd)

2.768571138381958


In [136]:
df.iloc[:30]

,item_id,vertical,category,domain,score
0,590602034,CE,ELECTRONICS,MLC-GAME_CONSOLES,0.9998
1,523534468,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9976
2,641416750,CE,MOBILE,MLC-CELLPHONES,0.9986
3,541283090,HOME & INDUSTRY,TOOLS AND CONSTRUCTION,MLC-ELECTRIC_DRILLS,0.9968
4,615879515,CE,MOBILE,MLC-CELLPHONES,0.9992
5,610865341,HOME & INDUSTRY,INDUSTRY,MLC-POINTS_OF_SALE_KITS,0.9960
6,908510601,CE,MOBILE,MLC-CELLPHONES,0.9988
7,611844936,BEAUTY & HEALTH,PHARMACEUTICS,MLC-SURGICAL_AND_INDUSTRIAL_MASKS,0.9884
8,582188629,HOME & INDUSTRY,HOME&DECOR,MLC-INDOOR_CURTAINS_AND_BLINDS,0.9822
9,538407191,HOME & INDUSTRY,HOME&DECOR,MLC-HOME_LIGHTING_SUPPLIES,0.9804
